In [2]:
import pandas as pd

In [3]:
df=pd.read_excel('simulated_x_monsoon120.xlsx')
df=df[['Date Time','WL (mMSL)','precip','month','year','Flash Flood','x']]

In [ ]:
df

,Date Time,WL (mMSL),precip,month,year,Flash Flood,x,x_base,x_total
0,2010-05-16 00:00:00,9.876000,0.600239,5,2010,0,0.000093,0.000093,0.000139
1,2010-05-16 01:00:00,9.863500,0.063596,5,2010,0,0.000196,0.000144,0.000268
2,2010-05-16 02:00:00,9.851000,0.036900,5,2010,0,0.000304,0.000198,0.000403
3,2010-05-16 03:00:00,9.838500,0.023337,5,2010,0,0.000416,0.000252,0.000542
4,2010-05-16 04:00:00,9.826000,0.128843,5,2010,0,0.000548,0.000311,0.000704
...,...,...,...,...,...,...,...,...,...
20323,2021-07-31 19:00:00,7.719167,0.466129,7,2021,0,0.757862,0.781402,1.187633
20324,2021-07-31 20:00:00,7.728333,0.417797,7,2021,0,0.754226,0.779587,1.182999
20325,2021-07-31 21:00:00,7.737500,0.645099,7,2021,0,0.745650,0.777782,1.173430
20326,2021-07-31 22:00:00,7.746667,0.956017,7,2021,0,0.738276,0.775996,1.165074


In [4]:
# ============================
# Optimize Monthly Constant Baseflow for Correlation
# ============================
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean()
    sd = s.std(ddof=0)
    return (s - mu) / (sd if sd else 1)

# Detect required columns/index
if 'x' not in df.columns:
    raise ValueError("Column 'x' not found in df. Compute UH first to create 'x'.")

# Try to find observed WL column
obs_candidates = ["WL (mMSL)", "WL", "Water level", "water_level"]
OBS_COL = None
for c in obs_candidates:
    if c in df.columns:
        OBS_COL = c
        break
if OBS_COL is None:
    raise ValueError(f"Observed column not found. Expected one of {obs_candidates}.")

# Use existing month column
if 'month' not in df.columns:
    raise ValueError("Column 'month' not found in df.")
month_series = pd.to_numeric(df['month'], errors="coerce")

# Build aligned pair (drop NaNs)
pair = pd.concat([df['x'], df[OBS_COL]], axis=1).dropna()
months_series = month_series.reindex(pair.index)
unique_months = sorted(months_series.dropna().astype(int).unique())

# Coordinate-descent over monthly constants
constants = {int(m): 0.0 for m in unique_months}

def corr_with_constants(constants_dict):
    x_adj = pair['x'].copy()
    for m, c in constants_dict.items():
        mask = months_series == m
        x_adj.loc[mask] = x_adj.loc[mask] + c
    return zscore(x_adj).corr(zscore(pair[OBS_COL]))

# Initialize
best_r = corr_with_constants(constants)

# Optimize per month using scipy bounded optimization for precision
for it in range(5):  # multiple passes for convergence
    for m in unique_months:
        # Define objective function to minimize (negative correlation)
        def objective(c):
            trial = dict(constants)
            trial[m] = c
            return -corr_with_constants(trial)  # negative because we minimize
        
        # Use bounded scalar minimization (constrained to non-negative)
        mask_m = months_series == m
        std_m = pair['x'].loc[mask_m].std(ddof=0)
        max_search = max(10 * std_m, 10.0)  # reasonable upper bound
        
        result = minimize_scalar(objective, bounds=(0, max_search), method='bounded')
        
        if result.success:
            constants[m] = result.x
            best_r = -result.fun  # convert back to positive correlation
        
print(f"Optimization converged. Best correlation: {best_r:.4f}")

# Apply constants to full df
df['baseflow_const'] = 0.0
for m, c in constants.items():
    df.loc[df['month'] == m, 'baseflow_const'] = c
df['x_total'] = df['x'] + df['baseflow_const']

# Final correlation on aligned pair
pair_total = pd.concat([df['x_total'], df[OBS_COL]], axis=1).dropna()
final_r = zscore(pair_total.iloc[:,0]).corr(zscore(pair_total.iloc[:,1]))

# Report results
summary_rows = [{"Month": int(m), "Constant": float(c)} for m, c in constants.items()]
summary_df = pd.DataFrame(summary_rows).sort_values('Month')
print("\n" + "="*60)
print("Monthly constant baseflow optimization summary")
print("="*60)
print(summary_df.to_string(index=False))
print(f"\nFinal z-score correlation (x_total vs {OBS_COL}): {final_r:.3f}")

# Optional: save summary
try:
    summary_df.to_csv("monthly_constant_baseflow_summary.csv", index=False)
    print("Saved: monthly_constant_baseflow_summary.csv")
except Exception as e:
    print(f"Could not save summary: {e}")

Optimization converged. Best correlation: 0.6552

Monthly constant baseflow optimization summary
 Month  Constant
     5  0.000004
     6  1.136815
     7  1.914914

Final z-score correlation (x_total vs WL (mMSL)): 0.655
Saved: monthly_constant_baseflow_summary.csv


In [5]:
# ============================
# Baseline Correlation (without baseflow constant)
# ============================
import numpy as np
import pandas as pd

def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean()
    sd = s.std(ddof=0)
    return (s - mu) / (sd if sd else 1)

# Check columns
if 'x' not in df.columns or 'WL (mMSL)' not in df.columns:
    print("Error: Required columns not found")
else:
    # Build aligned pair (drop NaNs)
    pair = pd.concat([df['x'], df['WL (mMSL)']], axis=1).dropna()
    
    # Compute baseline correlation (no baseflow adjustment)
    baseline_corr = zscore(pair['x']).corr(zscore(pair['WL (mMSL)']))
    
    print(f"Baseline z-score correlation (x vs WL without baseflow): {baseline_corr:.3f}")
    
    # Compare with optimized
    if 'x_total' in df.columns:
        pair_opt = pd.concat([df['x_total'], df['WL (mMSL)']], axis=1).dropna()
        optimized_corr = zscore(pair_opt['x_total']).corr(zscore(pair_opt['WL (mMSL)']))
        print(f"Optimized correlation (x_total vs WL with baseflow): {optimized_corr:.3f}")
        print(f"Improvement: {optimized_corr - baseline_corr:.3f} ({100*(optimized_corr - baseline_corr)/baseline_corr:.1f}%)")

Baseline z-score correlation (x vs WL without baseflow): 0.480
Optimized correlation (x_total vs WL with baseflow): 0.655
Improvement: 0.175 (36.6%)


In [ ]:
# ============================
# Plot Water Level, x, and x_total by Year
# ============================
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Get unique years
unique_years = sorted(df['year'].dropna().unique())

for year in unique_years:
    # Filter data for this year
    df_year = df[df['year'] == year].copy()
    
    # Create figure and primary axis
    fig, ax1 = plt.subplots(figsize=(14, 6))
    
    # Primary axis: Water Level
    color1 = 'tab:blue'
    ax1.set_xlabel('Date Time', fontsize=12)
    ax1.set_ylabel('Water Level (mMSL)', color=color1, fontsize=12)
    ax1.plot(df_year['Date Time'], df_year['WL (mMSL)'], color=color1, label='WL (mMSL)', linewidth=1.5)
    ax1.axhline(y=12.34, color='red', linestyle='--', linewidth=2, label='Threshold (12.34)')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Secondary axis: x and x_total
    ax2 = ax1.twinx()
    color2 = 'tab:orange'
    color3 = 'tab:green'
    ax2.set_ylabel('x and x_total', fontsize=12)
    ax2.plot(df_year['Date Time'], df_year['x'], color=color2, label='x', linewidth=1.2, alpha=0.7)
    ax2.plot(df_year['Date Time'], df_year['x_total'], color=color3, label='x_total (x + baseflow)', linewidth=1.2, alpha=0.7)
    ax2.tick_params(axis='y')
    ax2.legend(loc='upper right')
    
    # Format x-axis
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    fig.autofmt_xdate(rotation=45)
    
    # Title
    plt.title(f'Water Level, x, and x_total for Year {int(year)}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

print(f"\nGenerated plots for {len(unique_years)} years: {[int(y) for y in unique_years]}")